In [ ]:
'''Train a simple deep NN on the MNIST dataset.
Get to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import containers
from keras.layers.core import Dense, AutoEncoder, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils


batch_size = 64
nb_classes = 10
nb_epoch = 3

# the data, shuffled and split between tran and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

def plot_mnist_digit(image, figsize=None):
    """ Plot a single MNIST image."""
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    if figsize:
        ax.set_figsize(*figsize)
    ax.matshow(image, cmap = matplotlib.cm.binary)
    plt.xticks(np.array([]))
    plt.yticks(np.array([]))
    plt.show()

def plot_10_by_10_images(images, figsize=None):
    """ Plot 100 MNIST images in a 10 by 10 table. Note that we crop
    the images so that they appear reasonably close together.  The
    image is post-processed to give the appearance of being continued."""
    fig = plt.figure(figsize=figsize)
    #images = [image[3:25, 3:25] for image in images]
    #image = np.concatenate(images, axis=1)
    for x in range(10):
        for y in range(10):
            ax = fig.add_subplot(10, 10, 10*y+x)
            ax.matshow(images[10*y+x], cmap = matplotlib.cm.binary)
            plt.xticks(np.array([]))
            plt.yticks(np.array([]))
    plt.show()


In [ ]:
plot_mnist_digit(X_train[1])

In [ ]:
plot_10_by_10_images(X_train, figsize=(10,10))

In [ ]:
# the data, shuffled and split between train and test sets
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

In [ ]:
#creating the autoencoder
ae = Sequential()
encoder = containers.Sequential()
encoder.add(Dense(input_dim=784, output_dim=20, init='uniform'))
encoder.add(Activation('relu'))
decoder = containers.Sequential()
decoder.add(Dense(20, 784, init='uniform'))
decoder.add(Activation('relu'))
ae.add(AutoEncoder(encoder=encoder, decoder=decoder,
                   output_reconstruction=True))

In [ ]:
ae.compile(loss='mean_squared_error', optimizer=RMSprop())

In [ ]:
ae.fit(X_train, X_train, batch_size=batch_size, nb_epoch=nb_epoch,
       show_accuracy=True, verbose=1, validation_data=[X_test, X_test])